# Sentimental Text Analysis

In [137]:
import random

class Sentiment:
    NEGATIVE='NEGATIVE'
    NEUTRAL='NEUTRAL'
    POSITIVE='POSITIVE'
    
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
            
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self,reviews): #Method
        self.reviews=reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
        
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distributed(self):
        negative=list(filter(lambda x: x.sentiment==Sentiment.NEGATIVE, self.reviews))
        positive=list(filter(lambda x: x.sentiment==Sentiment.POSITIVE, self.reviews))
        #neutral=list(filter(lambda x: x.sentiment==Sentiment.NEUTRAL, self.reviews))
#To make negative values equal to positive values

        #print(negative[0].text)
        #print(len(negative))
        #print(len(positive))
        
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

In [138]:
#Since it's a raw text, only print statement won't work. Need to import json.
import json

file = r"C:\Users\priya\Desktop\Interview Prep\Books_small_10000.json"

reviews=[]
with open(file) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# Data Preparation

In [139]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(reviews, test_size=0.33, random_state=42)

#print(train[0].text)
train_container=ReviewContainer(train)
test_container=ReviewContainer(test)
cont.evenly_distributed()
len(cont.reviews)

872

In [140]:
train_container.evenly_distributed()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distributed()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


## Bag of words vectorization

In [150]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

I cannot believe this stuff,Kindle Amazon e books you should be ashamed.As with others I wasted my money.The prose is poor and the supposed plots are strung together with unhealthy porn(not good wholesome sex)the writers do not seem to be able to differentiate.I do not recommend
[[0. 0. 0. ... 0. 0. 0.]]


In [151]:
print(train_x[0])
print(train_x_vectors[0].toarray())

I cannot believe this stuff,Kindle Amazon e books you should be ashamed.As with others I wasted my money.The prose is poor and the supposed plots are strung together with unhealthy porn(not good wholesome sex)the writers do not seem to be able to differentiate.I do not recommend
[[0. 0. 0. ... 0. 0. 0.]]


# Classification

#### LINEAR SVM

In [152]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Decision Tree Classifier

In [153]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Naive Bayes

In [154]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)
clf_gnb.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [155]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Evaluation

In [156]:
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
#print(clf_gnb.score(test_x_vectors,test_y))
print(clf_log.score(test_x_vectors,test_y))

0.8076923076923077
0.6442307692307693
0.8052884615384616


In [157]:
# F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])
#f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE,  Sentiment.NEGATIVE])
#f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE,  Sentiment.NEGATIVE])
#f1_score(test_y, clf_gnb.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE,  Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [158]:
test_set = ['very fun', "bad book do not buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

#### Tuning our model (with Grid Search)

In [159]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [160]:
print(clf.score(test_x_vectors, test_y))

0.8197115384615384
